# Hugging Face 初體驗

Hugging Face 是一個專注於機器學習和自然語言處理的開源平台。它提供了大量的模型、數據集和工具，支持多種任務如文本生成、圖像分類和語音識別。用戶可以輕鬆地搜索、下載和使用這些資源來開發和部署AI應用。這個平台對研究人員和開發者來說非常有用，因為它促進了技術共享和協作。

平台提供各類開源模型及資料集，也針對不同性太的任務進行分類，便於查找使用．

![Hugging Face](./assets/huggingface.png)